In [ ]:
!pip install transformers
!pip install datasets
!pip install pykakasi

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ja")

train_dataset = dataset["train"]
train_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 6505
})

In [ ]:
train_dataset[0]

{'client_id': '3c9d94093b88b178118bb3be8de72d64c0423a0b8e32bfd799ce83e1e54249d8552205c1a9407bc11a3a0400d934b0277042caeb24e90b8087a83d884591e5b5',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/0fb75edb8bd454df8bb23ae8bee20532bae3434c4b63d0b4e126d443f35c56e7/ja_train_0/common_voice_ja_25861545.mp3',
  'array': array([ 0.00000000e+00, -9.94759830e-14,  3.19744231e-14, ...,
         -1.35150884e-07, -2.39865514e-08, -3.71753970e-08]),
  'sampling_rate': 48000},
 'sentence': '別の話を持ちかけられた。',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'male',
 'accent': '',
 'locale': 'ja',
 'segment': ''}

In [ ]:
train_dataset[0]['sentence']

'別の話を持ちかけられた。'

# 只獲取需要的 feature
## 只取 train_dataset的一部分，怕 colab跑不完

In [ ]:
# 選擇特定的行和列
# google colab上能力有限，等組員 local端建好，在跑完整 train_data
num_row = 111
data = train_dataset.select(range(num_row))
data

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 111
})

In [ ]:
selected_data = data.from_dict({
    'audio': data['audio'],
    'sentence': data['sentence']
})
selected_data

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 111
})

# 將 dataset轉換成期望的形式

In [ ]:
from pykakasi import kakasi

k = kakasi() # 初始化 kakasi
k.setMode('J', 'a')  # 從日文到羅馬拼音
k.setMode('K', 'a')  # 從片假名到羅馬拼音
k.setMode('H', 'a')  # 從平假名到羅馬拼音
k.setMode('r', 'Hepburn')  # 使用Hepburn羅馬拼音
k.setMode('s', True)  # 添加空格

def japanese_to_romaji(japanese):
  conv = k.getConverter()
  english = conv.do(japanese)
  return english.upper()

<ipython-input-7-5e3efd97a559>:4: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  k.setMode('J', 'a')  # 從日文到羅馬拼音
<ipython-input-7-5e3efd97a559>:5: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  k.setMode('K', 'a')  # 從片假名到羅馬拼音
<ipython-input-7-5e3efd97a559>:6: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  k.setMode('H', 'a')  # 從平假名到羅馬拼音
<ipython-input-7-5e3efd97a559>:7: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  k.setMode('r', 'Hepburn')  # 使用Hepburn羅馬拼音
<ipython-input-7-5e3efd97a559>:8: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  k.setMode('s', True)  # 添加空格


In [ ]:
import torchaudio

def process_waveforms(batch):

  waveform, sample_rate = torchaudio.load(batch['audio']['path'])

  if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

  # 如果 waveform 是雙聲道，需要轉單聲道。給 4GE用
  if waveform.size(0) > 1:
    waveform = waveform.mean(dim=0)

  # 讓 waveform的維度正確
  if waveform.ndim > 1:
    waveform = waveform.squeeze()

  batch["speech_array"] = waveform
  batch["sample_rate"] = 16000

  batch['labels'] = japanese_to_romaji(batch['sentence'])

  return batch

selected_data = selected_data.map(process_waveforms, remove_columns=selected_data.column_names)

selected_data

/usr/local/lib/python3.10/dist-packages/dill/_dill.py:412: PicklingWarning: Cannot locate reference to <enum '_TYPE'>.
  StockPickler.save(self, obj, save_persistent_id)
/usr/local/lib/python3.10/dist-packages/dill/_dill.py:412: PicklingWarning: Cannot pickle <enum '_TYPE'>: pykakasi.kakasi._TYPE has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

<ipython-input-7-5e3efd97a559>:11: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conv = k.getConverter()
<ipython-input-7-5e3efd97a559>:12: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  english = conv.do(japanese)


Dataset({
    features: ['speech_array', 'sample_rate', 'labels'],
    num_rows: 111
})

In [ ]:
# 單一的 speech_array要是1D list
# selected_data[0]['speech_array']

# Create HubertForCTC, Precessor

In [ ]:
from transformers import AutoProcessor, HubertForCTC

# 載入對應的 processor 和 model
model_name = "facebook/hubert-large-ls960-ft"
processor = AutoProcessor.from_pretrained(model_name)
model = HubertForCTC.from_pretrained(model_name)

model.config.ctc_zero_infinity = True

# 凍結 CNN 層參數
for name, param in model.named_parameters():
  if "Conv" in name:  # 假設 CNN 層參數名稱包含 'cnn'
    param.requires_grad = False
  if "conv" in name:
    param.requires_grad = False

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-

# Trainer

In [ ]:
import torch

class Trainer:
  def __init__(self, model, processor, optimizer):
    self.model = model
    self.processor = processor
    self.optimizer = optimizer
    self.total_loss = 0
    self.model.eval() # 不打這行，model也本來就預設evaluate mode

  def training_one_batch(self, batch):
    self.model.train()
    inputs = self.processor(batch["speech_array"],
                  sampling_rate=16000,
                  return_tensors="pt",
                  padding=True,
                  truncation=True,
                  max_length=160000)
    inputs["labels"] = self.processor(text=batch['labels'], return_tensors="pt", padding=True).input_ids

    # forward and predict & calculate CTC_loss

    outputs = self.model(**inputs)

    loss = outputs.loss
    self.total_loss += loss.item()

    # Backward and optimize
    self.optimizer.zero_grad()
    loss.backward()

    # 梯度裁切
    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

    self.optimizer.step()

    return inputs

  def evaluate(self, batch):
    self.model.eval()
    inputs = self.processor(batch["speech_array"],
                sampling_rate=16000,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=160000)
    inputs["labels"] = self.processor(text=batch['labels'], return_tensors="pt", padding=True).input_ids

    with torch.no_grad():
      outputs = self.model(**inputs)

    return outputs

In [ ]:
from torch import optim

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)

trainer = Trainer(model, processor, optimizer)

In [ ]:
# Training loop
num_epochs = 5
batch_size = 3

for epoch in range(num_epochs):
  trainer.total_loss = 0  # Reset total loss for each epoch
  num_batches = 0  # To count the number of batches processed

  # 應該要用 for batch in data_loader，但我不會，所以先用.map()
  # train時，出現的 WARNING，應該是 training_one_batch沒有從 class獨立出來的原因
  output = selected_data.map(trainer.training_one_batch, batched=True, batch_size=batch_size)
  print('total loss: ', trainer.total_loss)
  num_batches = num_row // batch_size
  # If the last batch is smaller than the batch size, account for it
  if num_row % batch_size != 0:
      num_batches += 1

  average_loss = trainer.total_loss / num_batches
  print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")


Parameter 'function'=<bound method Trainer.training_one_batch of <__main__.Trainer object at 0x7aa3b422db40>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

total loss:  10503.878036499023
Epoch 1/5, Average Loss: 283.8885955810547


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

total loss:  8565.34162902832
Epoch 2/5, Average Loss: 231.49571970346813


...

total loss:  7989.947128295898
Epoch 3/5, Average Loss: 215.94451698097023


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

total loss:  7686.421974182129
Epoch 4/5, Average Loss: 207.74113443735484


Map:   0%|          | 0/111 [00:00<?, ? examples/s]

total loss:  7237.067001342773
Epoch 5/5, Average Loss: 195.5964054416966


# 看一下從某個 sample預測出的文字

In [22]:
segment = 1
sample_batch = selected_data[segment]
output = trainer.evaluate(sample_batch)
output

CausalLMOutput(loss=tensor(27.6428), logits=tensor([[[  6.1055, -18.7336, -18.4877,  ...,  -4.5356,  -5.6195,  -4.5855],
         [  5.8916, -17.7170, -17.5449,  ...,  -4.5681,  -5.5733,  -4.9552],
         [  5.7055, -18.4084, -18.2579,  ...,  -4.3497,  -5.4134,  -4.5582],
         ...,
         [  5.1903, -17.6451, -17.7890,  ...,  -4.5180,  -6.1855,  -5.2718],
         [  5.4704, -18.1160, -18.1955,  ...,  -4.4754,  -6.2678,  -5.1221],
         [  5.6825, -18.4627, -18.4730,  ...,  -4.4728,  -6.2080,  -5.1818]]]), hidden_states=None, attentions=None)

In [23]:
logits = output.logits
predicted_ids = torch.argmax(logits, dim=-1)

In [24]:
print('predicted_ids: ', predicted_ids)
transcription = processor.batch_decode(predicted_ids)
print('predicted_text: ', transcription[0])
print('text_labels: ', selected_data[segment]['labels'])

predicted_ids:  tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 26,  0,  0,  8,
          0,  0,  0,  0, 10,  0,  0,  9,  0,  0,  8,  0,  0,  0,  0, 10,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
predicted_text:  KOINOI
text_labels:  OIOI。


In [21]:
from IPython.display import Audio

Audio(data[segment]['path'])

# Save Model

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
model_save_path = '/content/drive/My Drive/Colab_Notebooks/speech_research/Huert_base_on_English/hubert_finetuned.pth'
torch.save(model.state_dict(), model_save_path)

# Download Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [30]:
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

model.config.ctc_zero_infinity = True

# 凍結 CNN 層參數
for name, param in model.named_parameters():
  if "Conv" in name:  # 假設 CNN 層參數名稱包含 'cnn'
    param.requires_grad = False
  if "conv" in name:
    param.requires_grad = False

model_load_path = '/content/drive/My Drive/Colab_Notebooks/speech_research/Huert_base_on_English/hubert_finetuned.pth'
model.load_state_dict(torch.load(model_load_path))


Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-

<All keys matched successfully>